# 2. Coleta de dados

### Instalar/Importar bibliotecas

In [1]:
# ! pip install xlrd
# ! pip install openpyxl
# ! pip install pydantic
# ! pip install joblib

In [2]:
import pandas as pd
import os
import numpy as np
import yfinance as yf

from pydantic import BaseModel
from typing import Optional, List

import joblib

### Carregar dados de entrada

Classe para ler dados de balanco patrimonial e demonstrativo financeiro

In [3]:
class AquisicaoDadosFundamentos(BaseModel):
    balancos_dir: Optional[str] = "../dados/balancos/"
    balancos_joblib_file: Optional[str] = "../dados/fundamentos.joblib"
    fund_by_code: Optional[dict] = {}
    codes: List[str] = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        # Lista os arquivos xls na pasta dados/balancos
        files = os.listdir(self.balancos_dir)

        # Para cada arquivo (empresa) de balanco guarda no dicionario com o codigo como chave
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
    def salvar_joblib(self):
        joblib.dump(self.fund_by_code, self.balancos_joblib_file)



### Carregar dados de entrada

Classe para ler dados de cotacao historica das acoes

In [4]:
class AquisicaoDadosCotacoes(BaseModel):
    cotacoes_file: Optional[str] = "../dados/cotacoes/Cotacoes.xlsx"
    cotacoes_joblib_file: Optional[str] = "../dados/cotacoes.joblib"
    codes: List[str] = []
    cotacoes: Optional[dict] = {}

    def run(self):
        if os.path.isfile(self.cotacoes_joblib_file):
            print("carregando joblib cotacao")
            return joblib.load(self.cotacoes_joblib_file)

        self.get_cotacoes_from_excel()
        self.tratar_dados_nulos()
        self.salvar_joblib()
        return self.cotacoes

    def get_cotacoes_from_excel(self) -> None:
        """
        Le planilha com cotacoes historicas das acos do IBOV
        Para cada empresa, guarda no dicionario o dataframe com cotacoes e como chave o codigo da empresa
        """
        cotacoes_df = pd.read_excel(self.cotacoes_file)

        for empresa in cotacoes_df['Empresa'].unique():
            self.cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa']==empresa, :]
            
        print(len(self.cotacoes))

    def tratar_dados_nulos(self) -> None:
        """
        Remover empresas com cotacao contendo dados nulos
        """
        empresas_a_remover = []
        for empresa in self.codes:
            if self.cotacoes[empresa].isnull().values.any():
                self.cotacoes.pop(empresa)
                empresas_a_remover.append(empresa)
        self.codes = list(self.cotacoes.keys())
        print(len(self.codes))
        print(len(empresas_a_remover), empresas_a_remover, sep=" - ")

    def salvar_joblib(self):
        joblib.dump(self.cotacoes, self.cotacoes_joblib_file)



## Pré tratamento dos dados

### Remover empresas do dicionario que nao contem dados de cotacao historica

In [15]:
def remover_empresas_sem_dados_cotacao(fundamentos_by_code, cotacoes_by_code):
    fundamentos = fundamentos_by_code.copy()
    tamanho_inicial = len(fundamentos)
    # remove fundamentos das empresas que tenham cotacoes com dados nulos
    codes_to_be_removed_from_fund = list(set(fundamentos.keys()) ^ set(cotacoes_by_code.keys()))
    # print(codes_to_be_removed_from_fund)
    for code in codes_to_be_removed_from_fund:
        if code in fundamentos.keys():
            fundamentos.pop(code)

    if cotacoes_by_code.keys() == fundamentos.keys():
        print("Empresas sem dados de cotacoes removidos com sucesso")
        # print(len(fundamentos.keys()))

    print((tamanho_inicial - len(fundamentos)), "empresas removidas por nao terem dados cotacao")
    
    return fundamentos


### Juntar dicionarios de dataframes de dados financeiros com cotacoes

In [43]:
def juntar_fundamentos_com_cotacoes(fundamentos_by_code, cotacoes_by_code):
    """
    Trata os data frames de fundamentos e junta as cotacoes por trimestre
    """
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()
    for code in codes:
        if "Adj Close" not in fundamentos[code].columns:
            df = fundamentos[code].T
            df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
            # print(df)

            # Definir data como indice e pegar somente coluna de Adj Close do df
            if code in cotacoes_by_code.keys():
                df_cotacao = cotacoes_by_code[code].set_index("Date")
                df_cotacao = df_cotacao[["Adj Close"]]

                # Juntar dois dataframes
                df = df.merge(df_cotacao, right_index=True, left_index=True)
                df.index.name = code

                fundamentos[code] = df

        else:
            print("Tratamento ja executado")
    
    print("dataframes juntados com sucesso - ", len(fundamentos))
    return fundamentos


## Remocao de dados omissos/nulos

### Remover empresas com colunas diferentes da empresa referencia PETR4

In [7]:

def remover_empresa_colunas_diff(fundamentos_by_code):

    fundamentos = fundamentos_by_code.copy()
    tamanho_inicial = len(fundamentos)

    columns_ref = list(fundamentos["PETR4"].columns)
    """ 
    Remove empresas que nao tenham colunas de acordo com colunas de acao referencia 
    """
    codes = fundamentos.keys()
    empresa_a_remover = []
    for code in codes:
        if set(columns_ref) != set(fundamentos[code].columns):
            empresa_a_remover.append(code)
    
    for empresa in empresa_a_remover:
        fundamentos.pop(empresa)
    
    print((tamanho_inicial - len(fundamentos)), "empresas removidas por terem colunas diferentes de PETR4")

    return fundamentos




### Remover colunas que contem algum tipo de dado nulo ou zerado.

Algumas colunas aqui tambem foi verificado que eram dados repetidos

In [79]:
def remover_colunas_nulas(fundamentos_by_code):
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()

    colunas_remover = ["Receita Bruta de Vendas e/ou Serviços",
                        "Ativos Biológicos",
                        "Despesas Antecipadas",
                        "Deduções da Receita Bruta",
                        "Resultado Não Operacional",
                        "Receitas",
                        "Despesas",
                        "Diferido",
                        "Adiantamento para Futuro Aumento Capital",
                        "Passivos sobre Ativos Não-Correntes a Venda e Descontinuados",
                        "Lucros e Receitas a Apropriar",
                        "Reservas de Reavaliação",
                        "Adiantamento para Futuro Aumento Capital",
                        "Perdas pela Não Recuperabilidade de Ativos",
                        "Outras Receitas Operacionais",
                        "Participações/Contribuições Estatutárias",
                        "Reversão dos Juros sobre Capital Próprio"]
    for code in codes:
        try:
            for col in colunas_remover:
                if col in fundamentos[code].columns:
                    fundamentos[code].drop(columns=col, inplace=True)
            print(f"colunas nulas de {code} removidas com sucesso")
        except KeyError as e:
            print(f"empresa {code} sem as colunas a remover - err", code)
            raise e

    return fundamentos

### Remover empresas que mesmo apos remover colunas que nao serao utilizadas, contem dados nulos

In [78]:
def remover_empresas_com_valores_nulos(fundamentos_by_code):
    
    fundamentos = fundamentos_by_code.copy()

    columns = fundamentos["PETR4"].columns

    empresa_with_null = []

    for empresa in fundamentos:
        tabela = fundamentos[empresa]
        for coluna in columns:
            qtde_vazios = pd.isna(tabela[coluna]).sum()
            if isinstance(qtde_vazios, np.int64):
                if qtde_vazios > 0:
                    empresa_with_null.append(empresa)

    # remove empresas com valores nulos, ignorando ABEV por ter nulo na ultima linha
    for empresa in set(empresa_with_null):
        print(f"empresa {empresa} removida por ter valores nulos")
        fundamentos.pop(empresa)
    
    return fundamentos

### Obter dados historicos do indice Ibovespa para criacao da coluna decisao

In [11]:
def get_ibov_from_yahoo():

    if os.path.isfile("../dados/cotacoes_ibov.joblib"):
        print("carregando joblib ibov")
        return joblib.load("../dados/cotacoes_ibov.joblib")

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    return yf.download("^BVSP", start=data_inicial, end=data_final)


### Juntar dados historicos do Ibovespa ao dicionario de dataframes

In [12]:
def merge_ibov_by_fundamentos_dates(fundamentos_by_code, ibov):
    fundamentos = fundamentos_by_code.copy()
    datas_fundamentos = fundamentos["PETR4"].index

    # Set as nan when dates are different
    for data in datas_fundamentos:
        if data not in ibov.index:
            ibov.loc[data] = np.nan
    ibov = ibov.sort_index()
    ibov = ibov.ffill()
    ibov = ibov.rename(columns={"Adj Close": "IBOV"})

    for code in fundamentos:
        fundamentos[code] = fundamentos[code].merge(ibov[["IBOV"]], left_index=True, right_index=True)

    print(f"DF fundamentos e ibov juntados com sucesso - tamanho: {len(fundamentos)}")
    return fundamentos


### Salvar dataframe em joblib para proximo notebook

In [13]:
def salvar_dfs_in_joblib(df, file_name: str = "../out/result_aquisicao.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df unico criado com sucesso")


# Execucao

In [82]:
def main():

    # importar dados de balancos financeiros
    aquisicao_fund = AquisicaoDadosFundamentos()
    fundamentos_by_code = aquisicao_fund.run()

    # importar dados de cotacoes
    codes = list(fundamentos_by_code.keys())
    ad_cotacoes = AquisicaoDadosCotacoes(codes=codes)
    cotacoes_by_code = ad_cotacoes.run()

    print(f"Quantidade inicial de empresas: {len(fundamentos_by_code)}")

    fundamentos_by_code = remover_empresas_sem_dados_cotacao(fundamentos_by_code, cotacoes_by_code)
    fundamentos_by_code = juntar_fundamentos_com_cotacoes(fundamentos_by_code, cotacoes_by_code)


    # # tratar valores nulos e colunas
    fundamentos_by_code = remover_empresa_colunas_diff(fundamentos_by_code)
    fundamentos_by_code = remover_colunas_nulas(fundamentos_by_code)
    fundamentos_by_code = remover_empresas_com_valores_nulos(fundamentos_by_code)

    print(f"Quantidade final de empresas: {len(fundamentos_by_code)}")
    # # importar dados de cotacao do ibovespa
    ibov = get_ibov_from_yahoo()
    fundamentos_by_code = merge_ibov_by_fundamentos_dates(fundamentos_by_code, ibov)

    print(f"Quantidade final de empresas: {len(fundamentos_by_code)}")
    # # salvar em arquivo joblib
    # salvar_dfs_in_joblib(fundamentos_by_code)
    
    return fundamentos_by_code
    
bd = main()


bd["PETR4"].columns

carregando joblib fundamentos
carregando joblib cotacao
Quantidade inicial de empresas: 76
12 empresas removidas por nao terem dados cotacao
dataframes juntados com sucesso -  64
4 empresas removidas por terem colunas diferentes de PETR4
colunas nulas de MRFG3 removidas com sucesso
colunas nulas de ITSA4 removidas com sucesso
colunas nulas de CIEL3 removidas com sucesso
colunas nulas de PRIO3 removidas com sucesso
colunas nulas de TAEE11 removidas com sucesso
colunas nulas de JBSS3 removidas com sucesso
colunas nulas de MGLU3 removidas com sucesso
colunas nulas de PETR4 removidas com sucesso
colunas nulas de CYRE3 removidas com sucesso
colunas nulas de YDUQ3 removidas com sucesso
colunas nulas de GOLL4 removidas com sucesso
colunas nulas de SBSP3 removidas com sucesso
colunas nulas de MULT3 removidas com sucesso
colunas nulas de ECOR3 removidas com sucesso
colunas nulas de MRVE3 removidas com sucesso
colunas nulas de EZTC3 removidas com sucesso
colunas nulas de CCRO3 removidas com suce

Index(['Ativo Total', 'Ativo Circulante', 'Caixa e Equivalentes de Caixa',
       'Aplicações Financeiras', 'Contas a Receber', 'Estoques',
       'Tributos a Recuperar', 'Outros Ativos Circulantes',
       'Ativo Realizável a Longo Prazo',
       'Aplicações Financeiras Avaliadas a Valor Justo',
       'Aplicações Financeiras Avaliadas ao Custo Amortizado',
       'Contas a Receber', 'Estoques', 'Tributos Diferidos',
       'Créditos com Partes Relacionadas', 'Outros Ativos Não Circulantes',
       'Investimentos', 'Imobilizado', 'Intangível', 'Passivo Total',
       'Passivo Circulante', 'Obrigações Sociais e Trabalhistas',
       'Fornecedores', 'Obrigações Fiscais', 'Empréstimos e Financiamentos',
       'Passivos com Partes Relacionadas', 'Dividendos e JCP a Pagar',
       'Outros', 'Provisões', 'Passivo Não Circulante',
       'Empréstimos e Financiamentos', 'Passivos com Partes Relacionadas',
       'Outros', 'Tributos Diferidos', 'Provisões',
       'Participação dos Acionistas

In [83]:
bd["PETR4"].head(5)

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Tributos a Recuperar,Outros Ativos Circulantes,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo,...,Financeiras,Receitas Financeiras,Despesas Financeiras,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período,Adj Close,IBOV
2020-12-31,987418984.448,142323007.488,60856000.512,3424000,24583999.488,29500000.256,13482999.808,10476000.256,104974000.128,0,...,6811997.184,776999.936,6035002.368,74821001.216,-1498000.128,-12870999.04,-561999.936,59889999.872,27.418209,119306.0
2020-09-30,965430018.048,151643996.16,71635001.344,3782000.128,12904999.936,29789999.104,23851999.232,9680000,128209002.496,0,...,-22909999.104,667000,-23576999.936,-4878000.128,-26000,3235000.064,123000,-1546000,18.972162,94603.0
2020-06-30,1015142023.168,201926000.64,106597998.592,2952000,14316000.256,27594000.384,29788000.256,20678000.64,124448997.376,0,...,-12307999.744,579000,-12887000.064,-3288000,-610000,1084999.936,100000,-2712999.936,20.848644,95056.0
2020-03-31,971644993.536,163561996.288,80382001.152,3345999.872,15866000.384,31235999.744,13150000.128,19581999.104,119774003.2,0,...,-21177999.36,798000,-21976000.512,-66617999.36,-597000,17490999.296,1200999.936,-48523001.856,13.533956,73020.0
2019-12-31,926011031.552,112100999.168,29713999.872,3580000,15164000.256,33009000.448,14286999.552,16346999.808,71306002.432,0,...,-6862000.64,1258000,-8119999.488,6783998.976,2975000.576,-1698000.128,-385000,8152999.424,29.196196,115964.0


In [8]:
def remover_cols_com_todos_valores_zero(fundamentos_by_code):
    fundamentos = fundamentos_by_code.copy()

    null_cols_list = []
    for k in fundamentos.keys():
        df = fundamentos[k]

        # substituir todos os valores zeros por nan
        df.replace(0, np.nan, inplace=True)
        # colunas com todos valores nan
        null_cols = df.columns[df.isnull().all()]

        null_cols_list.extend(null_cols)

    cols_to_remove = set(null_cols_list)

    for k in fundamentos.keys():
        df = fundamentos[k]
        df.drop(cols_to_remove, axis = 1, inplace=True)
        df.drop(["Empréstimos e Financiamentos"], axis = 1, inplace=True)
        # replace nan by 1
        df.replace(np.nan, 1, inplace=True)
        fundamentos[k] = df

    return fundamentos